# What is Julia?

## "A New Approach to Technical Computing"

- Less than 10 years old - project started in 2012
- Growing use in scientific computing, data science, and machine learning
- Users in industry, government, and academia, all over the world

- One-sentence pitch: code as fast to write as Python, as fast to run as C++
- Actually, Julia can often be faster than C++
- Much easier to parallelize code across multiple cores, multiple nodes, or the GPU than Python
- Code is readable, easy to share with students, collaborators, and the public
- First class support for linear algebra and numerical computing
- The community is friendly and full of expertise from many domains
- Easy to learn if you already know Python
- Very good support for testing, documenting, and benchmarking code
- **Julia is one-based indexed, this isn't changing, love it or leave it.**

## Julia is used in a variety of fields

### Physics
- A group in CCQ, including Matt Fishman, Miles Stoudenmire, and me, has rewritten the C++ ITensors library in Julia and seen equivalent performance while adding features that would have been much harder to implement in C++ (like GPU support)
- Yao.jl, a state of the art quantum circuit simulator
- DiffEqPhysics


### Astronomy
- Celeste.jl
- Eric Agol's group in exoplanet astronomy
- https://github.com/marius311
- https://github.com/JuliaAstro/Cosmology.jl
- https://juliaastro.github.io/SkyCoords.jl/latest/#Performance-1
- https://github.com/JuliaAstro/LombScargle.jl#performance
- https://github.com/JuliaAstro/Photometry.jl

### Biology
- BioJulia packages
- BioSequences.jl performance blogpost (https://biojulia.net/post/seq-lang/)
- DiffEqBiological

### Climate Science
- Clima.jl
- Oceananigans


## Drawbacks of using Julia

- Since the language is new, many people are unfamiliar with it and unwilling to switch
- Less well developed community than Python has - no "JuliaLadies", for example, or regional JuliaCons
- Onboarding and tutorial materials still lacking - a great opportunity for someone to get involved
- Fewer mature packages than Python or C++
- Tooling is less mature -- debuggers, IDEs, etc (this is improving quickly)

## Just how fast is this language?

We can look at two kinds of benchmarks: "micro", for small operations that are repeated many times (like `sqrt`) and "macro", for a complete simulation/data analysis.

![micro-benchmarks](https://julialang.org/assets/benchmarks/benchmarks.svg)

## Some simple examples of Julia code

- A `for` loop
- Broadcasting
- Defining custom types
- Multiple dispatch
- Type instability!

In [1]:
# There are lots of ways to iterate through a collection

A = rand(512)
B = similar(A)
@show eltype(A)
@show typeof(B)
for i in 1:length(A)
    B[i] = sin(A[i])
end

for (i, aa) in enumerate(A) # probably recognize this from Python
    B[i] = sin(aa)
end

for i in 2:2:length(A) # let's slice!
    B[i] = cos(A[i])
end

Ar = reshape(A, (256, 2))
Br = reshape(B, (2, 256))

# can still use linear indices
B[:] = A[:]

# we can fill B using a "generator" (like a list comprehension)
B = [sin(a) for a in A]

# or we can even use broadcasting
B .= sin.(A)

eltype(A) = Float64
typeof(B) = Array{Float64,1}


512-element Array{Float64,1}:
 0.8225715825963349
 0.479279124544149
 0.7369790489376848
 0.04424066072168864
 0.359034435164221
 0.540565357663336
 0.35429971136774074
 0.6503484133912293
 0.627763802552588
 0.2980414085099836
 0.32971548857104127
 0.3845157061744127
 0.1453505036682908
 ⋮
 0.2979773267075211
 0.7358774273037118
 0.10509494464785275
 0.08280422371918808
 0.16093486947169752
 0.11557731098201743
 0.8047861429069003
 0.7691141267000563
 0.8250530173416858
 0.36109722006657907
 0.1965099645821133
 0.7118489365789462

### More on broadcasting and memory

Because Julia is memory-managed, it's better if we can avoid allocations in our code, especially in tight inner loops. We can use the basic `@time` macro to see if some code we wrote is allocating.

In [2]:
@time B = copy(A) # allocating

@time B = sin.(A)

@time B .= sin.(A)

  0.001068 seconds (28 allocations: 6.047 KiB)
  0.012362 seconds (37.72 k allocations: 1.826 MiB)
  0.000011 seconds (2 allocations: 32 bytes)


512-element Array{Float64,1}:
 0.8225715825963349
 0.479279124544149
 0.7369790489376848
 0.04424066072168864
 0.359034435164221
 0.540565357663336
 0.35429971136774074
 0.6503484133912293
 0.627763802552588
 0.2980414085099836
 0.32971548857104127
 0.3845157061744127
 0.1453505036682908
 ⋮
 0.2979773267075211
 0.7358774273037118
 0.10509494464785275
 0.08280422371918808
 0.16093486947169752
 0.11557731098201743
 0.8047861429069003
 0.7691141267000563
 0.8250530173416858
 0.36109722006657907
 0.1965099645821133
 0.7118489365789462

Broadcasting is a nice way to minimize the number of allocations in code, and Julia can *automatically* broadcast functions over arrays and custom types. Let's look at some examples and introduce Julia's type system.

In [3]:
# define a custom *abstract* type
abstract type AbstractPoint{T, N} end

mutable struct BPoint{N, T} <: AbstractPoint{N, T}
    coords::NTuple{N,T} # a Tuple of N elements of type T
end

# let's construct some BPoints

my_2d_point = BPoint{2, Int}(1, 2)

my_3d_point = BPoint{3, Float64}(1.0, 2.0, 3.0)

MethodError: MethodError: no method matching BPoint{2,Int64}(::Int64, ::Int64)
Closest candidates are:
  BPoint{2,Int64}(::Any) where {N, T} at In[3]:5

The above didn't work, because we need to define a constructor for `Point`:

In [4]:
mutable struct Point{N, T} <: AbstractPoint{N, T}
    coords::NTuple{N,T} # a Tuple of N elements of type T
    function Point{N, T}(coords::Vararg{T, N}) where {N, T}
        new(coords)
    end
end

# let's construct some Points

my_2d_point = Point{2, Int}(1, 2)

my_3d_point = Point{3, Float64}(1.0, 2.0, 3.0)


Point{3,Float64}((1.0, 2.0, 3.0))

In [5]:
@show my_2d_point isa Point
@show my_2d_point isa Point{2}
@show my_2d_point isa Point{Int}
@show my_2d_point isa Point{3, Int}

my_2d_point isa Point = true
my_2d_point isa Point{2} = true
my_2d_point isa Point{Int} = false
my_2d_point isa Point{3, Int} = false


false

What if we don't know the coordinate type in advance?

In [6]:
try
    my_2d_point = Point(1, 2)
catch e
    @warn e
end

Point(coords::Vararg{T, N}) where {N, T} = Point{N, T}(coords...)

my_2d_point = Point(1, 2)

┌ Warning: MethodError(Point, (1, 2), 0x0000000000006bc3)
└ @ Main In[6]:4


Point{2,Int64}((1, 2))

### Why is this useful?

We can write custom methods for our types!

In [7]:
function angle(point_a, point_b)
    θ_a = atan(point_a.coords[2]/point_a.coords[1])
    θ_b = atan(point_b.coords[2]/point_b.coords[1])
    θ_b - θ_a
end

# let's write some tests for this!
using Test
y_axis = Point(0, 1)
x_axis = Point(1, 0)
@test angle(x_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == -π/2

Test Passed

This *seems* to work ... but what if I have a 3d point? Can't I define an angle between two 3-d rays? (What do the astronomers think about this idea?). In fact, we know that:

`a * b = |a||b|cosθ`

In [8]:
function angle(point_a, point_b)
    inner_prod = sum(point_a.coords .* point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle(x_axis, y_axis) == π/2
@test angle(z_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == π/2

# and let's run some timings

Test Passed

This method shouldn't work if we pass points with different dimensionality - one way to deal with this would be:

In [9]:
Base.ndims(p::Point{N, T}) where {N, T} = N
using LinearAlgebra
function angle_notype(point_a, point_b)
    ndims(point_a) == ndims(point_b) || throw(DimensionMismatch("A and B have different dimensionality"))
    inner_prod = dot(point_a.coords, point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle(x_axis, y_axis) == π/2
@test angle(z_axis, y_axis) == π/2
@test angle(y_axis, x_axis) == π/2

@test_throws DimensionMismatch angle(Point(1, 2), Point(1, 2, 3))

Test Passed
      Thrown: DimensionMismatch

In [10]:
using LinearAlgebra
function angle_typed(point_a::Point{N, T}, point_b::Point{N, T}) where {N, T}
    inner_prod = dot(point_a.coords, point_b.coords)
    mag_a = hypot(point_a.coords...)
    mag_b = hypot(point_b.coords...)
    acos(inner_prod/(mag_a*mag_b))
end
y_axis = Point(0, 1, 0)
x_axis = Point(1, 0, 0)
z_axis = Point(0, 0, 1)
@test angle_typed(x_axis, y_axis) == π/2
@test angle_typed(z_axis, y_axis) == π/2
@test angle_typed(y_axis, x_axis) == π/2
@test_throws MethodError angle_typed(Point(1, 2), Point(1, 2, 3))

as = [Point(a...) for a in collect(zip(rand(512), rand(512)))]
bs = [Point(b...) for b in collect(zip(rand(512), rand(512)))]

@time angle_notype.(as, bs)
@time angle_typed.(as, bs)

  0.088700 seconds (285.72 k allocations: 14.192 MiB)
  0.045539 seconds (142.23 k allocations: 6.833 MiB)


512-element Array{Float64,1}:
 0.24621051401695263
 0.35135114147039553
 1.0254872278322142
 0.49568426422207235
 0.0037863315297453266
 0.1851987227050544
 0.39121530613781835
 0.02639813355058079
 0.015332570125683032
 0.7104044759305768
 0.4546063244767796
 0.47332233426295606
 0.4259838014796647
 ⋮
 0.5542326963203039
 0.2983918290253858
 0.6868174311841276
 0.6225687473176721
 0.3538490265861798
 0.6850302217658225
 1.2172107629192448
 0.8490943775052646
 1.1141425365714168
 0.3766414808701807
 0.3174457002450253
 0.46791915114526866

## Multiple dispatch: an example with our points

In [11]:
@show as isa Vector

# define a new "scalar multiplication" method for Point

Base.:*(x::Number, a::Point) = Point((x .* a.coords)...)
Base.:*(a::Point, x::Number) = Point((x .* a.coords)...)
Base.:+(a::Point{N, T}, b::Point{N, T}) where {N, T} = Point((a.coords .+ b.coords)...)

cs = [Point(c...) for c in collect(zip(rand(512), rand(512)))]
cns = rand(512)

ds = cns .* cs

es = Point.(rand(12, 12))
fs = rand(12, 12)

es * fs

as isa Vector = true


MethodError: MethodError: no method matching zero(::Point{1,Float64})
Closest candidates are:
  zero(!Matched::Type{LibGit2.GitHash}) at /Users/khyatt/julia/master/usr/share/julia/stdlib/v1.5/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeight}) at /Users/khyatt/julia/master/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/versionweights.jl:15
  zero(!Matched::Type{Missing}) at missing.jl:103
  ...

In [13]:
# We need to define what the zero of this type is
Base.zero(::Type{Point{N, T}}) where {N, T} = Point(ntuple(i->zero(T), N)...)
Base.zero(p::Point{N, T}) where {N, T}      = Point(ntuple(i->zero(T), N)...)

es * fs

12×12 Array{Point{1,Float64},2}:
 Point{1,Float64}((2.71569,))  …  Point{1,Float64}((2.22517,))
 Point{1,Float64}((2.50973,))     Point{1,Float64}((1.90612,))
 Point{1,Float64}((2.92649,))     Point{1,Float64}((2.34365,))
 Point{1,Float64}((4.60811,))     Point{1,Float64}((3.4951,))
 Point{1,Float64}((3.42013,))     Point{1,Float64}((2.49952,))
 Point{1,Float64}((2.68227,))  …  Point{1,Float64}((2.04708,))
 Point{1,Float64}((4.03045,))     Point{1,Float64}((3.92484,))
 Point{1,Float64}((2.31827,))     Point{1,Float64}((2.17938,))
 Point{1,Float64}((3.52559,))     Point{1,Float64}((2.09282,))
 Point{1,Float64}((2.69568,))     Point{1,Float64}((2.50669,))
 Point{1,Float64}((2.96938,))  …  Point{1,Float64}((2.83372,))
 Point{1,Float64}((3.94182,))     Point{1,Float64}((3.38907,))

In [15]:
Base.:/(a::Point, x::Number) = Point((inv(x) .* a.coords)...)
es ./ 2.0

12×12 Array{Point{1,Float64},2}:
 Point{1,Float64}((0.142765,))   …  Point{1,Float64}((0.0597799,))
 Point{1,Float64}((0.0052819,))     Point{1,Float64}((0.096078,))
 Point{1,Float64}((0.0965424,))     Point{1,Float64}((0.399058,))
 Point{1,Float64}((0.0676617,))     Point{1,Float64}((0.443464,))
 Point{1,Float64}((0.110137,))      Point{1,Float64}((0.424328,))
 Point{1,Float64}((0.0804803,))  …  Point{1,Float64}((0.401861,))
 Point{1,Float64}((0.440812,))      Point{1,Float64}((0.310611,))
 Point{1,Float64}((0.424045,))      Point{1,Float64}((8.70159e-6,))
 Point{1,Float64}((0.147607,))      Point{1,Float64}((0.479501,))
 Point{1,Float64}((0.194041,))      Point{1,Float64}((0.298807,))
 Point{1,Float64}((0.41732,))    …  Point{1,Float64}((0.0385975,))
 Point{1,Float64}((0.0319677,))     Point{1,Float64}((0.490056,))

## It's easy to use libraries written in other languages

This way, you can use Julia for parts of the simulation you want to rewrite now, while still retaining the battle-tested library you've put many hours into developing.

### From Python

Julia has a very nice package called `PyCall` that lets you import Python libraries, use Python types, and call Python functions. Let's look at an example:

In [ ]:
using PyCall, Conda
Conda.add("geopy", channel="conda-forge")
gp = pyimport("geopy")
using JLD, FileIO
coords = jldopen("coord_df.jld", "r") do file
    read(file, "coords")
end
gcs = pyimport("geopy")."geocoders"
locator = gcs.Nominatim(user_agent="citibike", timeout=nothing)
gx = pyimport("geopy.extra.rate_limiter")
rl = gx.RateLimiter(locator.reverse, min_delay_seconds=1.1, max_retries=4)

located = coords |> Tables.transform(Dict(:coord=>(x->rl(x)))) |> DataFrame

### From C

- Calling C from Julia (or Julia from C!) is natively supported, as is using C `struct`s, and all those good things. Julia and C can pass arrays back and forth, and it's simple to call any function in a shared object library from Julia.

### From FORTRAN

## Showcases

In this section we'll see some cool examples of things Julia and its packages are able to do:
- Random walks - +
- Benchmark tools and testsets - 
- Profiling -
- Interactive plotting - +
- Multi-node job handling
  - `ClusterManagers.jl` - +
  - `WorkerGroup`
- Parallelism
  - Threads - show atomics, maybe a lock?
  - `pmap` - 2D array example - +
- Differential equations solving (Schroedinger equation)
  - https://github.com/SciML/DiffEqTutorials.jl
- Native Julia machine learning - +
  - Flux
  - GeometricFlux
  - Transformers

## Resources to learn more

- There is a Julia book!
- Julia slack is very welcoming and we have dedicated channels for physics, astronomy, biology, and areas like HPC
- We also have a discourse forum for people to ask questions and get help
- The Julia youtube page has many talks from previous JuliaCons, which often include Jupyter notebook tutorials
- Learn X in Y minutes - a basic introduction to Julia syntax